# EXAMINATION OF VEHICLE EMISSIONS (Data Cleaning Exploration)

This notebook compiles publicly available datasets used for analyzing emissions from passenger electric vehicles (EVs)  and internal combustion engine (ICE) vehicles.

## IMPORTS

In [1]:
import numpy as np
import pandas as pd
import path
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# Included to support accessing the variables and dataframes in this notebook from another notebook
import os
from datetime import datetime, timedelta

In [3]:
# Included to ignore any warning dialoge generated
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Added to show all columns in df display
pd.options.display.max_columns = None 

## GENERAL VEHICLE DATA

**VEH DATASET 1: Historic Vehicle Sales (1976 to 2023)** \
_Federal Reserve Economic Data, FRED , Light Weight Vehicle Sales in the US (thousand units) and Total Vehicle Sales in the US (million units) \
Documentation available here: https://fredaccount.stlouisfed.org/datalists/324414_

In [5]:
# Reads in seasonally adjusted US vehicle sales data
total_veh_sales_SA = pd.read_csv("Resources/TOTALSA.csv", parse_dates=['DATE'])
light_veh_sales_SA = pd.read_csv("Resources/ALTSALES.csv", parse_dates=['DATE'])
light_veh_truck_sales_SA = pd.read_csv("Resources/LTRUCKSA.csv", parse_dates=['DATE'])
heavy_veh_truck_sales_SA = pd.read_csv("Resources/HTRUCKSSAAR.csv", parse_dates=['DATE'])

total_veh_sales_SA = (total_veh_sales_SA
                      .merge(light_veh_sales_SA, on='DATE', how='left')
                      .merge(light_veh_truck_sales_SA, on='DATE', how='left')
                      .merge(heavy_veh_truck_sales_SA, on='DATE', how='left')
                     )
                      


total_veh_sales_SA = total_veh_sales_SA.rename(columns={'TOTALSA': 'Total Vehicle Sales',
                                                        'ALTSALES': 'Light Veh',
                                                        'LTRUCKSA': 'Light Veh - Truck',
                                                        'HTRUCKSSAAR': 'Heavy Veh'
                                                       })

total_veh_sales_SA ['Light Veh - Car'] = total_veh_sales_SA ['Light Veh'] - total_veh_sales_SA ['Light Veh - Truck'] 

# Reorering columns
reindexed = ['DATE', 'Total Vehicle Sales', 'Light Veh', 'Light Veh - Car', 'Light Veh - Truck', 'Heavy Veh']

# Reorder the columns
total_veh_sales_SA = total_veh_sales_SA.reindex(columns = reindexed)

# Aggregates monthly sales data into annual sales data
total_veh_sales_SA = total_veh_sales_SA.set_index('DATE')
annual_veh_sales_SA = total_veh_sales_SA.resample('Y').sum()
annual_veh_sales_SA = annual_veh_sales_SA.reset_index()
annual_veh_sales_SA['DATE'] = annual_veh_sales_SA['DATE'].dt.year


annual_veh_sales_SA['Light Veh Share - Car (%)'] = annual_veh_sales_SA['Light Veh - Car'] / annual_veh_sales_SA['Light Veh']
annual_veh_sales_SA['Light Veh Share - Truck (%)'] = annual_veh_sales_SA['Light Veh - Truck'] / annual_veh_sales_SA['Light Veh']

#drops 2024 data (partial year)
annual_veh_sales_SA.drop(48, inplace = True)

# Display the first few rows of the result
display('annual_veh_sales_SA (million units), 1976 to 2023') 
display(annual_veh_sales_SA.head(5))

'annual_veh_sales_SA (million units), 1976 to 2023'

,DATE,Total Vehicle Sales,Light Veh,Light Veh - Car,Light Veh - Truck,Heavy Veh,Light Veh Share - Car (%),Light Veh Share - Truck (%)
0,1976,159.479,155.593,119.932,35.661,3.886,0.770806,0.229194
1,1977,178.370,173.829,132.551,41.278,4.541,0.762537,0.237463
2,1978,184.958,179.674,133.967,45.707,5.284,0.745611,0.254389
3,1979,169.898,165.216,126.706,38.510,4.682,0.766911,0.233089
4,1980,137.563,134.309,107.782,26.527,3.254,0.802493,0.197507


**VEH DATASET 2: Fuel Economy Standards by Year (1978-2031)** \
_Corporate Average Fuel Economy (CAFE) standards in miles per gallon (mpg) provided by National Highway Traffic Safety Administration
and US EPA \
Documentation available at: https://afdc.energy.gov/data/10562_

In [6]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
vehicle_emission_standards = pd.read_csv("Resources/vehicle_efficiency_CAFE_requirements.csv")

# Display the first 5 rows
vehicle_emission_standards.fillna(0,inplace=True)

# Display the first 5 rows
display('vehicle_emission_standards (mpg, 1978 to 2031)') 
display(vehicle_emission_standards.head(5))

'vehicle_emission_standards (mpg, 1978 to 2031)'

,Model Year,Passenger Cars,Light-Duty Trucks
0,1978,18.0,0.0
1,1979,19.0,0.0
2,1980,20.0,0.0
3,1981,22.0,0.0
4,1982,24.0,17.5


**VEH DATASET 3: Real-World Vehicle Fuel Economy and Emissions** \
_2023 EPA Automotive Trends Report data in US for light duty fuel efficiency standards from 1975 to 2023\
Provides data for supporting fuel efficency predictions above minimum standards and beyond 2031 \
Documentation available here: https://www.epa.gov/automotive-trends/about-automotive-trends-data_

In [7]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
real_world_emissions = pd.read_csv("Resources/estimated_real_world_fuel_economy (1975 to 2022).csv", parse_dates=['Model Year'])

# Replaces missing values with 0 and filters for all vehicle emissions
real_world_emissions.fillna(0,inplace=True)

#Filters emissions for all (aggreggated) type and drops unndeded columns
real_world_emissions_all = real_world_emissions[(real_world_emissions['Vehicle Type'] == 'All')].copy().reset_index(drop=True)

real_world_emissions.drop(columns = ['Production Share', 'Real-World MPG_City', 'Real-World MPG_Hwy',
                                         'Real-World CO2_City (g/mi)', 'Real-World CO2_Hwy (g/mi)',
                                         'Weight (lbs)','Horsepower (HP)', 'Footprint (sq. ft.)'],
                                        axis = 1, inplace = True)

#Removes text from 'Model Year' column
def clean_year(year):
    if isinstance(year, str) and 'Prelim.' in year:
        return year.split()[-1]  # Extract the year from 'Prelim. 2023'
    return str(year)  # Convert all years to string for consistency

real_world_emissions['Model Year'] = real_world_emissions['Model Year'].apply(clean_year)

# Simplifes dataset by consoildating MPG and emissions by Car Class, Truck Class and All (Cars and Trucks)
real_world_emissions['Class'] = real_world_emissions['Regulatory Class'].replace({'Car': 'Car', 'Truck': 'Truck', 'All': 'All'})

emissions_by_class = real_world_emissions.groupby(['Model Year', 'Class']).agg({
    'Real-World MPG': 'mean',
    'Real-World CO2 (g/mi)': 'mean'
}).reset_index()

emissions_by_class = emissions_by_class.sort_values(['Model Year', 'Class'])

#Converts 'Model Year' to datetime datatype
emissions_by_class['Model Year'] = pd.to_datetime(emissions_by_class['Model Year'], format='%Y')
emissions_by_class['Model Year'] = emissions_by_class['Model Year'].dt.year

emissions_by_class.tail(10)


,Model Year,Class,Real-World MPG,Real-World CO2 (g/mi)
137,2020,Truck,22.165925,403.809827
138,2021,All,25.424540,346.851700
139,2021,Car,31.693723,273.445910
140,2021,Truck,23.342683,385.885353
141,2022,All,25.993490,337.081030
142,2022,Car,33.281957,255.597267
143,2022,Truck,23.323518,382.958172
144,2023,All,26.941060,319.989890
145,2023,Car,35.224507,235.984623
146,2023,Truck,24.185080,364.592482


In [43]:
# Builds Emissions by Class Dataset

# Modifies original emissions table to expand all values to single Model Year line
emissions_reshaped = emissions_by_class.pivot(index='Model Year', 
                                              columns='Class', 
                                              values=['Real-World MPG', 'Real-World CO2 (g/mi)'])

emissions_reshaped.columns = [f'{val} ({col.lower()})' for val, col in emissions_reshaped.columns]
emissions_reshaped = emissions_reshaped.reset_index()

column_order = ['Model Year', 
                'Real-World MPG (all)', 'Real-World CO2 (g/mi) (all)',
                'Real-World MPG (car)', 'Real-World CO2 (g/mi) (car)',
                'Real-World MPG (truck)', 'Real-World CO2 (g/mi) (truck)']
emissions_reshaped = emissions_reshaped[column_order]

#Adds the future emission standards values to the historic real-world emissions data
emissions_and_stds_by_class = pd.merge(emissions_reshaped, vehicle_emission_standards, on='Model Year', how='outer')

def fill_future_values(row):
    if row['Model Year'] >= 2024 and row['Model Year'] <= 2035:
        if pd.isna(row['Real-World MPG (car)']):
            row['Real-World MPG (car)'] = row['Passenger Cars']
        if pd.isna(row['Real-World MPG (truck)']):
            row['Real-World MPG (truck)'] = row['Light-Duty Trucks']
    return row

emissions_and_stds_by_class = emissions_and_stds_by_class.apply(fill_future_values, axis=1)

# Drop the original vehicle_emission_standards columns
emissions_and_stds_by_class = emissions_and_stds_by_class.drop(['Passenger Cars', 'Light-Duty Trucks'], axis=1)

# Sort by Model Year
emissions_and_stds_by_class = emissions_and_stds_by_class.sort_values('Model Year')

# Reset index
emissions_and_stds_by_class =emissions_and_stds_by_class.reset_index(drop=True)

# Rename columns
emissions_and_stds_by_class = emissions_and_stds_by_class.rename(columns={
    'Real-World MPG (car)': 'MPG (car)',
    'Real-World MPG (truck)': 'MPG (light truck)',
    'Real-World MPG (all)': 'MPG (all)',
    'Real-World CO2 (g/mi) (car)': 'CO2 (g/mi) (car)',
    'Real-World CO2 (g/mi) (truck)': 'CO2 (g/mi) (light truck)',
    'Real-World CO2 (g/mi) (all)': 'CO2 (g/mi) (all)'
})

# Reorder columns
column_order = ['Model Year', 
                'MPG (all)', 'CO2 (g/mi) (all)',
                'MPG (car)', 'CO2 (g/mi) (car)',
                'MPG (light truck)', 'CO2 (g/mi) (light truck)']

emissions_and_stds_by_class = emissions_and_stds_by_class[column_order]

# Fill NaN values with 0
emissions_and_stds_by_class = emissions_and_stds_by_class.fillna(0)

# Round values to 2 decimal places
emissions_and_stds_by_class = emissions_and_stds_by_class.round(2)

# Convert 'Model Year' to integer first, then to datetime
emissions_and_stds_by_class['Model Year'] = pd.to_datetime(emissions_and_stds_by_class['Model Year'].astype(int).astype(str) + '-01-01')

# Display the dataframe with 'Model Year' formatted to show only the year
emissions_and_stds_by_class['Model Year'] = emissions_and_stds_by_class['Model Year'].dt.strftime('%Y')

display('emissions_and_stds_by_class (MPG, CO2 g/mi)')
display('real-world emissions from 1975 to 2023 and future emissions standards from 2024 to 2031')
display(emissions_and_stds_by_class.tail (10))


'emissions_and_stds_by_class (MPG, CO2 g/mi)'

'real-world emissions from 1975 to 2023 and future emissions standards from 2024 to 2031'

,Model Year,MPG (all),CO2 (g/mi) (all),MPG (car),CO2 (g/mi) (car),MPG (light truck),CO2 (g/mi) (light truck)
47,2022,25.99,337.08,33.28,255.60,23.32,382.96
48,2023,26.94,319.99,35.22,235.98,24.19,364.59
49,2024,0.00,0.00,54.50,0.00,37.50,0.00
50,2025,0.00,0.00,58.90,0.00,39.30,0.00
51,2026,0.00,0.00,64.80,0.00,42.40,0.00
52,2027,0.00,0.00,66.10,0.00,42.40,0.00
53,2028,0.00,0.00,65.20,0.00,43.20,0.00
54,2029,0.00,0.00,72.50,0.00,54.90,0.00
55,2030,0.00,0.00,80.10,0.00,62.10,0.00
56,2031,0.00,0.00,91.60,0.00,70.80,0.00


**VEH DATASET 4: Energy consumption of full electric vehicles** \
_Based on real-world values with averages corrected for multiple versions of the same model.\
As of July 2024, aAverag BEV energy consumption is 303 watt-hours/mile and average batter range is 236 miles.\
Documentation available at: https://ev-database.org/imp/cheatsheet/energy-consumption-electric-car_

In [38]:
avg_veh_yrs = 16
avg_mi_per_yr = 13000
mi_under_warranty = 100000
life_mi = avg_veh_yrs * avg_mi_per_yr

display(f"Avg passenger vehicle drive miles per year (avg_mi_per_yr): {avg_mi_per_yr}")
display(f"Avg passenger vehicle miles over useful life (life_mi): {life_mi}")
display(f"Avg passenger vehicle miles under warranty (mi_under_warranty): {mi_under_warranty}")
display(f"Avg passenger vehicle useful life (avg_veh_yrs): {avg_veh_yrs} years")

'Avg passenger vehicle drive miles per year (avg_mi_per_yr): 13000'

'Avg passenger vehicle miles over useful life (life_mi): 208000'

'Avg passenger vehicle miles under warranty (mi_under_warranty): 100000'

'Avg passenger vehicle useful life (avg_veh_yrs): 16 years'

## ELECTRIC VEHICLE DATA

**EV DATASET 1: Historical EV Stock (2013 to 2023)** \
_Contains share (in millions of units) of EV stock in US, Europe, China, and rest of world from 2013 to 2023
Documentation available at: https://www.iea.org/data-and-statistics/charts/global-electric-car-stock-2013-2023_


In [9]:
# Imports historic US BEV/PHEV stock data
BEV_PHEV_stock= pd.read_csv("Resources/global-electric-car-stock-2013-2023.csv",
                                     delimiter =';',
                                     skiprows=3,
                                     header=0)

BEV_PHEV_stock = BEV_PHEV_stock.rename(columns={'Unnamed: 0': 'Year'})

drop_col = ['China BEV', 'China PHEV', 'Europe BEV', 'Europe PHEV', 'Rest of the world BEV', 'Rest of the world PHEV'] 
BEV_PHEV_stock.drop(columns = drop_col, axis = 1, inplace = True)

display('BEV_PHEV_stock (millions of units), 2013 to 2023')
display(BEV_PHEV_stock.head(5))

'BEV_PHEV_stock (millions of units), 2013 to 2023'

,Year,United States BEV,United States PHEV
0,2013,0.1,0.1
1,2014,0.1,0.2
2,2015,0.2,0.2
3,2016,0.3,0.3
4,2017,0.4,0.4


**EV DATASET 2: Historic and Projected (2011 to 2035) EV Sales by Powertrain** \
_Includes historical and projected data aligned to stated policies scenario (STEPS) and announced pledges scenario (APS) \
for electric vehicles sales, stock, charging infrastructure and oil displacement \
Documentation available at: https://www.iea.org/data-and-statistics/data-product/global-ev-outlook-2024#global-ev-data_

In [10]:
# Reads in historic and projeted EV electric demand data and joins datafames
ev_outlook= pd.read_csv("Resources/IEA Global EV Data 2024.csv")

# Historic EV Outlook data for US
ev_outlook_US_hist = ev_outlook[(ev_outlook['category'] == 'Historical') & (ev_outlook['region'] == 'USA') & (ev_outlook['mode'] == 'Cars')].copy().reset_index(drop=True)

# Projected EV Outlook data based on stated policies for US
ev_outlook_US_proj_STEPS = ev_outlook[(ev_outlook['category'] == 'Projection-STEPS') & (ev_outlook['region'] == 'USA') & (ev_outlook['mode'] == 'Cars')].copy().reset_index(drop=True)

# Projected EV Outlook data based on announced pledges for US
ev_outlook_US_proj_APS = ev_outlook[(ev_outlook['category'] == 'Projection-APS') & (ev_outlook['region'] == 'USA') & (ev_outlook['mode'] == 'Cars')].copy()

display('ev_outlook_US_hist (millions of units, %), 2011 to 2035')
display(ev_outlook_US_hist)

display('ev_outlook_US_proj_STEPS (millions of units, %), 2011 to 2035')
display(ev_outlook_US_proj_STEPS)

'ev_outlook_US_hist (millions of units, %), 2011 to 2035'

,region,category,parameter,mode,powertrain,year,unit,value
0,USA,Historical,EV sales,Cars,BEV,2010,Vehicles,1200.00000
1,USA,Historical,EV stock,Cars,BEV,2010,Vehicles,3800.00000
2,USA,Historical,Oil displacement Mbd,Cars,EV,2010,Milion barrels per day,0.00016
3,USA,Historical,"Oil displacement, million lge",Cars,EV,2010,"Oil displacement, million lge",9.40000
4,USA,Historical,Electricity demand,Cars,EV,2010,GWh,96.00000
...,...,...,...,...,...,...,...,...
139,USA,Historical,EV sales,Cars,FCEV,2023,Vehicles,3000.00000
140,USA,Historical,EV stock,Cars,FCEV,2023,Vehicles,18000.00000
141,USA,Historical,EV sales,Cars,PHEV,2023,Vehicles,290000.00000
142,USA,Historical,"Oil displacement, million lge",Cars,EV,2023,"Oil displacement, million lge",9700.00000


'ev_outlook_US_proj_STEPS (millions of units, %), 2011 to 2035'

,region,category,parameter,mode,powertrain,year,unit,value
0,USA,Projection-STEPS,Oil displacement Mbd,Cars,EV,2020,Milion barrels per day,0.043
1,USA,Projection-STEPS,"Oil displacement, million lge",Cars,EV,2020,"Oil displacement, million lge",2500.000
2,USA,Projection-STEPS,Electricity demand,Cars,EV,2020,GWh,5400.000
3,USA,Projection-STEPS,EV sales,Cars,BEV,2020,Vehicles,230000.000
4,USA,Projection-STEPS,EV stock,Cars,BEV,2020,Vehicles,1100000.000
...,...,...,...,...,...,...,...,...
72,USA,Projection-STEPS,Oil displacement Mbd,Cars,EV,2035,Milion barrels per day,2.500
73,USA,Projection-STEPS,"Oil displacement, million lge",Cars,EV,2035,"Oil displacement, million lge",140000.000
74,USA,Projection-STEPS,Electricity demand,Cars,EV,2035,GWh,460000.000
75,USA,Projection-STEPS,EV stock share,Cars,EV,2035,percent,36.000


**EV DATASET 4: Electricity Consumption (MWh) by Light-Duty Electric Vehicles (2018 to Q1 2024)** \
_Includes electricty consumption by BEVs and PHEVs in the light-duty vehicle (LDV) class.\
LDVs are passenger cars and light trucks weighing less than 8,500 lbs.\n.
Documentation available at: https://www.eia.gov/electricity/monthly/epm_table_grapher.php?t=table_d_1_

In [11]:
url = 'https://www.eia.gov/electricity/monthly/epm_table_grapher.php?t=table_d_1'
tables = pd.read_html(url)
LDV_elec_use = tables[1]

# Function extracts 'Year' from 'Period' column
def extract_year(period):
    if isinstance(period, str):
        if period.startswith('Year '):
            return int(period.split()[-1])
        elif period.isdigit():
            return int(period)
    elif isinstance(period, (int, float)):
        return int(period)
    return None

LDV_elec_use['Year'] = LDV_elec_use['Period'].apply(extract_year)

# Converts data in table to numberic values
for col in ['Plug-in Hybrid Electric Vehicle (PHEV)', 'Battery Electric Vehicle (BEV)', 'Total']:
    LDV_elec_use[col] = pd.to_numeric(LDV_elec_use[col], errors='coerce')


# Aggregates data by year 
LDV_elec_by_year = LDV_elec_use.dropna(subset=['Year']).groupby('Year').agg({
    'Plug-in Hybrid Electric Vehicle (PHEV)': 'sum',
    'Battery Electric Vehicle (BEV)': 'sum',
    'Total': 'sum'
}).reset_index()
 

# Simplifies column headers
LDV_elec_by_year = LDV_elec_by_year.reset_index(drop=True)

LDV_elec_by_year = LDV_elec_by_year.rename(columns={
    'Plug-in Hybrid Electric Vehicle (PHEV)': 'PHEV',
    'Battery Electric Vehicle (BEV)': 'BEV'})


# Converts 'Year' datetime datetype and sorts by Year
LDV_elec_by_year[col] = LDV_elec_by_year[col].astype('int64')
LDV_elec_by_year['Year'] = pd.to_datetime(LDV_elec_by_year['Year'].astype(int).astype(str)).dt.strftime('%Y')
LDV_elec_by_year = LDV_elec_by_year.sort_values('Year')




display('LDV_elec_by_year (MWh), 2018 to Q1-2024')
display(LDV_elec_by_year.head(10))


'LDV_elec_by_year (MWh), 2018 to Q1-2024'

,Year,PHEV,BEV,Total
0,2018,756806.0,824899.0,1581706
1,2019,884161.0,1175714.0,2059875
2,2020,1073251.0,1827049.0,2900300
3,2021,1242674.0,2276123.0,3518797
4,2022,2168276.0,4617155.0,6785431
5,2023,2792300.0,6980868.0,9773167
6,2024,900346.0,2348595.0,3248941


**EV DATASET 5: Energy consumption of full electric vehicles** \
_Based on real-world values with averages corrected for multiple versions of the same model.\
As of July 2024, aAverag BEV energy consumption is 303 watt-hours/mile and average batter range is 236 miles.\
Documentation available at: https://ev-database.org/imp/cheatsheet/energy-consumption-electric-car_

In [35]:
kwh_per_mile = 303/1000
mi_per_charge = 236
life_chg_cyc = life_mi/mi_per_charge 
war_chg_cyc = mi_under_warranty/mi_per_charge 


print(f"Average BEV kWh conumption per mile (kwh_per_mile): {kwh_per_mile}")
print(f"Average BEV miles per full battery (mi_per_charge): {mi_per_charge}")
print(f"Average number of full charge cycles under BEV warrenty(war_chg_cyc): {war_chg_cyc:.00f}")
print(f"Average number of full charge cycles over BEV lifetime(life_chg_cyc): {life_chg_cyc:.00f}")

Average BEV kWh conumption per mile: 0.303
Average BEV miles per full battery: 236
Average number of full charge cycles under BEV warrenty:  424
Average number of full charge cycles over BEV lifetime: 881


## ELECTRICITY DATA

**ELEC DATASET 1: US Electricity Generation By Fuel Type** \
_EIA net electricity generation for all sectors energy in US from 2010 to 2023 \
Documentation available at: link to be added_

In [12]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
net_generation = pd.read_csv("Resources/Net_generation_for_all_sectors.csv",
                                   skiprows=4,
                                   header=0)

# Replaces missing values with 0
net_generation.fillna(0,inplace=True)

# Drops uneeded columns
net_generation.drop(columns="source key", axis=1, inplace=True)
net_generation['description'] = net_generation['description'].str.replace('United States : ', '', regex=False)
net_generation = net_generation.iloc[2:]
net_generation = net_generation.reset_index(drop=True)
net_generation = net_generation.set_index('description')
net_generation.index.name = None

# Transposes dataframe to show records by year to align with other datasets
net_generation = net_generation.T
net_generation = net_generation.reset_index()
net_generation = net_generation.rename(columns={'index': 'year'})
net_generation.drop(0, inplace=True)

# Display the first 5 rows
display('net_generation (thousand MWh), 2010 to 2023')
display (net_generation.head(5))

'net_generation (thousand MWh), 2010 to 2023'

,year,all fuels (utility-scale),coal,petroleum liquids,petroleum coke,natural gas,other gases,nuclear,conventional hydroelectric,other renewables,wind,all utility-scale solar,geothermal,biomass,wood and wood-derived fuels,other biomass,hydro-electric pumped storage,other,all solar,small-scale solar photovoltaic,all utility-scale solar
1,2010,4125060,1847290,23337,13724,987697,11313,806968,260203,0,94652,1212,15219,56089,37172,18917,-5501,12855,--,--,1212
2,2011,4100141,1733430,16086,14096,1013689,11566,790204,319355,0,120177,1818,15316,56671,37449,19222,-6421,14154,--,--,1818
3,2012,4047765,1514043,13403,9787,1225894,11898,769331,276240,0,140822,4327,15562,57622,37799,19823,-4950,13787,--,--,4327
4,2013,4065964,1581115,13820,13344,1124836,12853,789016,268565,0,167840,9036,15775,60858,40028,20830,-4681,13588,--,--,9036
5,2014,4093564.0,1581710.0,18276.0,11955.0,1126635.0,12022.0,797166.0,259367.0,0.0,181655.0,17691.0,15877.0,63989.0,42340.0,21650.0,-6174.0,13393.0,28924.0,11233.0,17691.0


**ELEC DATASET 2: US Electricity Consumption By Fuel Type** \
_EIA electricity consumption for all energy sectors in US from 2010 to 2023\
Documentation available at: link to be added_

In [13]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
sector_elec_consumption = pd.read_csv("Resources/Consumption_for_electricity_generation_for_all_sectors.csv",
                                   skiprows=4,
                                   header=0)

# Replaces missing values with 0
sector_elec_consumption.fillna(0,inplace=True)

# Removes uneeded data
sector_elec_consumption.drop(columns="source key", axis=1, inplace=True)
sector_elec_consumption['description'] = sector_elec_consumption['description'].str.replace('United States : ', '', regex=False)
sector_elec_consumption = sector_elec_consumption.iloc[2:]
sector_elec_consumption = sector_elec_consumption.reset_index(drop=True)
sector_elec_consumption = sector_elec_consumption.set_index('description')
sector_elec_consumption.index.name = None

# Transposes dataframe to show records by year to align with other datasets
sector_elec_consumption = sector_elec_consumption.T
sector_elec_consumption = sector_elec_consumption.reset_index()
sector_elec_consumption = sector_elec_consumption.rename(columns={'index': 'year'})
sector_elec_consumption.drop(0, inplace=True)

# Display the first 5 rows
display('sector_elec_consumption (thousand tons coal, thousand barrels liquid, thousand tons coke, thousand Mcf NG), 2010 to 2023')
display(sector_elec_consumption.head(5))

'sector_elec_consumption (thousand tons coal, thousand barrels liquid, thousand tons coke, thousand Mcf NG), 2010 to 2023'

,year,coal,petroleum liquids,petroleum coke,natural gas
1,2010,979684.0,40103.0,4994.0,7680185.0
2,2011,934938.0,27326.0,5012.0,7883865.0
3,2012,825734.0,22604.0,3675.0,9484710.0
4,2013,860729.0,23231.0,4852.0,8596299.0
5,2014,853634.0,31531.0,4412.0,8544387.0


## US GHG Emissions Datasets

**GHG DATASET 1: Compariative Lifecycle GHG emission for Midsized BEV and ICE vehicle**
_Compares life-cycle greenhouse gas emissions of a mid-size BEV to ICE vehicle \
Considers vehicle manufacturing, battery assembly, battery mineral mining, electricity and fuel cycle well-to-wheels emissions \
Data measued in tCO2e per vehicle lifetime and sourced from IEA / Argonne National Lab analysis conducted 2020 \
Documentation available at: https://www.iea.org/data-and-statistics/charts/comparative-life-cycle-greenhouse-gas-emissions-of-a-mid-size-bev-and-ice-vehicle_

In [14]:
# Reads in lifecycle emissions data for BEV and ICE
veh_lifecycle_GHG = pd.read_csv("Resources/comparative-life-cycle-greenhouse-gas-emissions-of-a-mid-size-bev-and-ice-vehicle.csv",
                                     skiprows=3,
                                     header=0)

veh_lifecycle_GHG = veh_lifecycle_GHG.rename(columns={'Unnamed: 0': 'Vehicle Type'})
veh_lifecycle_GHG.drop(columns='Unnamed: 6', axis = 1, inplace=True)
veh_lifecycle_GHG['Embodied CO2e'] = veh_lifecycle_GHG['Vehicle manufacturing'] + veh_lifecycle_GHG['Batteries-assembly and other'] + veh_lifecycle_GHG['Batteries-minerals'] 
veh_lifecycle_GHG['Operations CO2e'] = veh_lifecycle_GHG['Electricity'] + veh_lifecycle_GHG['Electricity'] + veh_lifecycle_GHG['Fuel cycle (well-to-wheel)']
veh_lifecycle_GHG['Lifecycle CO2e'] = veh_lifecycle_GHG['Embodied CO2e'] + veh_lifecycle_GHG['Operations CO2e'] 

display('veh_lifecycle_GHG (tCO2e per vehicle), 2020 estimate')
display(veh_lifecycle_GHG)

'veh_lifecycle_GHG (tCO2e per vehicle), 2020 estimate'

,Vehicle Type,Vehicle manufacturing,Batteries-assembly and other,Batteries-minerals,Electricity,Fuel cycle (well-to-wheel),Embodied CO2e,Operations CO2e,Lifecycle CO2e
0,Battery electric vehicle - Base case,5.4,1.20,1.40,11.7,0.0,8.00,23.4,31.40
1,Battery electric vehicle - High-GHG minerals ...,5.4,1.20,2.80,11.7,0.0,9.40,23.4,32.80
2,Internal combustion engine vehicle,6.0,0.01,0.03,0.0,35.9,6.04,35.9,41.94


**GHG DATASET 2: Global Average Vehicle Lifecycle Emissions by Power Train (2023 and 2035)** \
Compares global average lifecycle emissions (t CO2e/vehicle) by powertrain based on APS and STEPS, 2023-2035\
GHG from electricity production not included.
Documentation available at: https://www.iea.org/data-and-statistics/charts/comparison-of-global-average-lifecycle-emissions-by-powertrain-in-the-stated-policies-and-announced-pledges-scenarios-2023-2035

In [15]:
# Reads in lifecycle emissions data for each pawertrain type (CO2-eq/vehicle)
veh_lifecycle_GHG_2035= pd.read_csv("Resources/comparison-of-global-average-lifecycle-emissions-by-powertrain-in-the-stated-policies-and-announced-pledges-scenarios-2023-2035.csv",
                                    delimiter=";", 
                                     skiprows=3,
                                     header=0)

veh_lifecycle_GHG_2035 = veh_lifecycle_GHG_2035.rename(columns={'Unnamed: 0': 'Powertrain Type'})
veh_lifecycle_GHG_2035 = veh_lifecycle_GHG_2035.drop(range(8, 12))
veh_lifecycle_GHG_2035['Year'] = '2035'
veh_lifecycle_GHG_2035['Year'][range(0,4)] = '2023'

veh_lifecycle_GHG_2035['Embodied CO2e'] = veh_lifecycle_GHG_2035['Car production'] + veh_lifecycle_GHG_2035['Battery production'] + veh_lifecycle_GHG_2035['Well-to-tank'] 
veh_lifecycle_GHG_2035['Tailpipe CO2e'] = veh_lifecycle_GHG_2035['Tank-to-wheel']
veh_lifecycle_GHG_2035['Lifecycle CO2e'] = veh_lifecycle_GHG_2035['Embodied CO2e'] + veh_lifecycle_GHG_2035['Tailpipe CO2e'] 

# Reording columns
col_order = ['Year', 'Powertrain Type', 'Car production', 'Battery production',
             'Well-to-tank', 'Tank-to-wheel', 'Embodied CO2e', 'Tailpipe CO2e', 
             'Grid decarbonisation impact', 'Lifecycle CO2e']

veh_lifecycle_GHG_2035 = veh_lifecycle_GHG_2035[col_order]

veh_lifecycle_GHG_2023 = veh_lifecycle_GHG_2035.drop(range(4,8))
veh_lifecycle_GHG_2035 = veh_lifecycle_GHG_2035.drop(range(0,4))
veh_lifecycle_GHG_2035.reset_index(drop = True, inplace = True)


display('veh_lifecycle_GHG_2035 (tCO2e per vehicle), 2035 projection')
display(veh_lifecycle_GHG_2023.head(10))
display(veh_lifecycle_GHG_2035.head(10))


'veh_lifecycle_GHG_2035 (tCO2e per vehicle), 2035 projection'

,Year,Powertrain Type,Car production,Battery production,Well-to-tank,Tank-to-wheel,Embodied CO2e,Tailpipe CO2e,Grid decarbonisation impact,Lifecycle CO2e
0,2023,ICEV,3.7,0.0,11.5,30.9,15.2,30.9,0.0,46.1
1,2023,HEV,3.7,0.1,9.8,26.3,13.6,26.3,0.0,39.9
2,2023,PHEV,4.4,1.3,11.7,15.8,17.4,15.8,1.9,33.2
3,2023,BEV,3.3,5.3,14.5,0.0,23.1,0.0,4.8,23.1


,Year,Powertrain Type,Car production,Battery production,Well-to-tank,Tank-to-wheel,Embodied CO2e,Tailpipe CO2e,Grid decarbonisation impact,Lifecycle CO2e
0,2035,ICEV,3.7,0.0,8.1,26.4,11.8,26.4,0.0,38.2
1,2035,HEV,3.7,0.1,6.9,22.4,10.7,22.4,0.0,33.1
2,2035,PHEV,4.4,0.8,7.7,13.4,12.9,13.4,4.2,26.3
3,2035,BEV,2.9,3.2,8.8,0.0,14.9,0.0,10.6,14.9


**GHG DATASET 3: Marginal Emission Rates**\
_U.S. national weighted average CO2 marginal emission rate, year 2021 data\
Documentation available at: https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references_

In [41]:
#Average US (based on 2021 values)
gCO2_per_kWh = 700
mmtCO2_kWh_consumed = .000417
mmtCO2_MWh_consumed = .417
mmtCO2_kWh_saved = .000699
mmtCO2_MWh_saved = .699

print(f"mmtCO2 emissions from consuming a kWh of electricity in US: {mmtCo2_kWh_consumed} or {mmtCo2_MWh_consumed} mmtCO2 per MWh")
print(f"mmtCO2 emissions from conserving a kWh of electricity in US: {mmtCO2_kWh_saved} or {mmtCO2_MWh_saved} mmtCO2 per MWh")


mmtCO2 emissions from consuming a kWh of electricity in US: 0.000417 or0.417 mmtCO2 per MWh
mmtCO2 emissions from conserving a kWh of electricity in US: 0.000699 or0.699 mmtCO2 per MWh


**GHG DATASET 4: Emissions by Sector** \
_EPA US Greenhouse Gas Inventory by Economic Sector, MMT CO2 eq.\
Documentation available at https://afdc.energy.gov/data/10802_

In [16]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
emissions_by_econ_sector = pd.read_csv("Resources/GHG_emissions_by_econ_sector.csv",
                                       skiprows=2,
                                       header=0)
                                      
# Drop unnamed columns
emissions_by_econ_sector = emissions_by_econ_sector.drop(columns=[col for col in emissions_by_econ_sector.columns if col.startswith('Unnamed')])

# Converts data columns to numeric
columns_to_convert = ['Transportation', 'Electricity Generation', 'Industry', 'Total']
emissions_by_econ_sector[columns_to_convert] = emissions_by_econ_sector[columns_to_convert].replace(',', '', regex=True).astype(float)

# Display the first 5 rows and data types
display('emissions_by_econ_sector (MMT CO2e), 1990 to 2022')
display(emissions_by_econ_sector.head(5))

'emissions_by_econ_sector (MMT CO2e), 1990 to 2022'

,Year,Transportation,Electricity Generation,Industry,Agriculture,Commercial,Residential,Total
0,1990,1521.4,1880.2,1723.3,595.9,447.0,345.6,6513.5
1,1991,1474.8,1874.4,1702.9,587.4,454.5,355.3,6449.2
2,1992,1533.8,1889.6,1729.3,587.5,450.0,361.8,6552.0
3,1993,1570.2,1965.0,1701.0,608.7,443.1,373.1,6661.2
4,1994,1624.5,1989.6,1719.7,612.2,446.1,363.8,6755.8


**GHG DATASET 4: Net Avoided Emissions and Avoided Share (2023 to 2035)** \
_IEA projection of net avoided emissions from BEV and ICE using lifecycle GHG values (measured in millions of tons CO2e)\
and share of net avoided emissions (measured in percentages) for stated policies scenario (STEPS), announced pledges scenario (APS) \
and Net Zero Emissions (NZE) \
Documentation available at: https://www.iea.org/reports/global-ev-outlook-2024/outlook-for-emissions-reductions#abstract_

In [17]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
avoided_emissions_2035_proj = pd.read_csv("Resources/net-avoided-well-to-wheel-greenhouse-gas-emissions-from-electric-vehicle-deployment-2023-2035.csv",
                                          delimiter=';',
                                          skiprows=3,
                                          header=0)

avoided_emissions_2035_proj = avoided_emissions_2035_proj.rename(columns={'Unnamed: 0': 'Year'})


display('avoided_emissions_2035_proj (MMT CO2e), 2023 to 2035')
display(avoided_emissions_2035_proj.head(10))

'avoided_emissions_2035_proj (MMT CO2e), 2023 to 2035'

,Year,STEPS,APS,NZE
0,2023,-130,-130,-120
1,2024,-180,-180,-200
2,2025,-240,-250,-310
3,2026,-320,-330,-440
4,2027,-420,-430,-600
5,2028,-540,-550,-790
6,2029,-680,-700,-1000
7,2030,-840,-880,-1230
8,2031,-1010,-1070,-1470
9,2032,-1190,-1270,-1720


## Dataset Use

**EXTRACTING DATAFRAMES FOR USE** \
The %run magic command may be added to other .ipynb files to execute the this notebook

In [18]:
# EXECUTED FROM NEW NOTEBOOK OPTION
    # Runs all the code in vehicle_fuel_emissions_data.ipynb, its variables and dataframes available in other notebook
    # To use this option, paste the "%run vehicle_fuel_emissions_data" command into a notebook that will execute this notebook

# %run vehicle_fuel_emissions_data.ipynb 